In [1]:
import os, sys
print(os.listdir('../../'))
print(os.getcwd())
sys.path.append("../")
sys.path.append("../src/network")
sys.path.append("../src")
print(os.getcwd())
import torch
import glob
import string
import os, sys
import cv2
import numpy as np
from network.gen_model.gen_model import GenModel_FC
from random import choices
import random
import matplotlib.pyplot as plt
from preproc import preproc_iam
from torchvision import transforms
from tokenizer import Tokenizer
import multiprocessing


['.git', '.gitignore', 'data', 'fid_folder', 'gen_val_data.ipynb', 'plot_loss.ipynb', 'raw', 'README.md', 'siamese_src', 'small_experiments', 'src', 'src_semi', 'subset', 'subset_gen.ipynb', 'TTUR-master.zip', 'word_len_counts.png']
c:\Users\Lisa\Documents\GitHub\handwriting-recognition\siamese_src\data
c:\Users\Lisa\Documents\GitHub\handwriting-recognition\siamese_src\data


In [2]:
charset_base = string.ascii_lowercase + string.ascii_uppercase
max_text_length = 25
tokenizer = Tokenizer(chars=charset_base, max_text_length=max_text_length)

### Loading generator model

In [3]:
gen_model = GenModel_FC(tokenizer.maxlen, tokenizer.vocab_size, tokenizer.PAD).cuda()
gen_model.load_state_dict(torch.load('../../src/network/gen_model/gen_model-2750-25ch-corr-half.model')) #load
gen_model.eval()

GenModel_FC(
  (enc_image): RecursiveScriptModule(
    original_name=ImageEncoder
    (model): RecursiveScriptModule(
      original_name=VGG
      (features): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=Conv2d)
        (1): RecursiveScriptModule(original_name=InstanceNorm2d)
        (2): RecursiveScriptModule(original_name=ReLU)
        (3): RecursiveScriptModule(original_name=Conv2d)
        (4): RecursiveScriptModule(original_name=InstanceNorm2d)
        (5): RecursiveScriptModule(original_name=ReLU)
        (6): RecursiveScriptModule(original_name=Conv2d)
        (7): RecursiveScriptModule(original_name=InstanceNorm2d)
        (8): RecursiveScriptModule(original_name=ReLU)
        (9): RecursiveScriptModule(original_name=Conv2d)
        (10): RecursiveScriptModule(original_name=InstanceNorm2d)
        (11): RecursiveScriptModule(original_name=ReLU)
        (12): RecursiveScriptModule(original_name=MaxPool2d)
        (13):

### global variables

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
input_size = (64, 216, 1)
num_style_imgs = 15 # num imgs for generator to extract style from
batch_size = 32
dataset = "iam_gan"
dataset_path = os.path.join("..", "..", "data", dataset, "words")
max_word_len = 10

### Preprocessing data

In [ ]:
# preproc_bool = False
# if preproc_bool:
#     path_from = os.path.join("..", "..", "..", "handwriting-recognition", "raw", dataset)
#     os.makedirs(dataset_path, exist_ok=True)
#     preproc_iam(path_from, dataset_path)

### Reading in the data

In [6]:
def read_data(folder, max_word_len):
    """Get data paths and labels (with max_word_len) of images in folder."""
    dataset = {}
    wid_dict = {}
    lens = {}
    dataset = []
    wid_dict = {}

    text_files = [os.path.join(folder, f"ground_truth_train_filtered.txt"), os.path.join(folder, f"ground_truth_valid_filtered.txt")]

    for text_file in text_files:
        with open(text_file, encoding='utf-8') as data_file:
            lines = data_file.read().splitlines()

        for line in lines:
            line_split = line.split(' ')
            wid = None

            if not set(string.digits).isdisjoint(set(line_split[-1])):
                wid = line_split[-1]
                line_split = line_split[:-1]
                
            if len(line_split) > 2:
                img_path, gt_label = line_split[0], ''.join(line_split[1:])
            else:
                img_path, gt_label = line_split[0], line_split[1]
                
            if len(gt_label) > max_word_len:
                continue

            if len(gt_label) in lens.keys():
                lens[len(gt_label)] += 1
            else:
                lens[len(gt_label)] = 1
            
            img_path = img_path.replace("/", "\\")

            if wid is not None:
                if wid not in wid_dict.keys():
                    wid_dict[wid] = []
                
                wid_dict[wid].append((img_path, gt_label))

            # img_path = os.path.join(folder, partition, img_path)
            dataset.append((img_path, gt_label, wid))
        print(f"number of words: {len(dataset)}")
        print(f"number of wids: {len(wid_dict.keys())}")

        lens = dict(sorted(lens.items()))

        print(f"Number of words per word length")
        for key in lens.keys():
            print(key, lens[key])

    return dataset, wid_dict

In [8]:
data, wids_dict = read_data(dataset_path, max_word_len)

wids = [tup[2] for tup in data]
print(data[0])

print(len(data))

number of words: 22117
number of wids: 339
Number of words per word length
2 4817
3 5670
4 4367
5 2981
6 2242
7 2040
number of words: 31295
number of wids: 500
Number of words per word length
2 6677
3 8172
4 6304
5 4242
6 3109
7 2791
('..\\data\\iam_gan\\words\\train\\a03-034-00-00.png', 'Members', '049')
31295


### Unique words in dataset

In [ ]:
# unique_word_counts = []
# all_words = []

# word_len_dict = {}

# for wid in wids.keys():
#     words = list(set([wids[wid][idx][1] for idx in range(len(wids[wid]))]))
#     all_words += words
#     unique_word_counts.append(len(words))
# unique_word_counts = sorted(unique_word_counts, reverse=True)
# len(set(all_words))

# for word in set(all_words):
#     if len(word) not in word_len_dict.keys():
#         word_len_dict[len(word)] = []
#     word_len_dict[len(word)].append(word)


5% of unique words to validation
5% of unique words to test

### Generate validation, test, and training splits

First 80/20 train/test
then 80/20 train/valid


In [9]:
from sklearn.model_selection import train_test_split

sup_val_fn = "../../data/iam_gan/words/superHTR.ground_truth_val_filtered.txt"
sup_tr_fn = "../../data/iam_gan/words/superHTR.ground_truth_train_filtered.txt"
sup_te_fn = "../../data/iam_gan/words/superHTR.ground_truth_test_filtered.txt"


x_train, x_test, wids_train, wids_test = train_test_split(data, wids, test_size=0.3, random_state=42, stratify=wids, shuffle=True)
print(len(x_train), len(x_test), len(wids_train), len(wids_test))

x_val = []
x_new_test = []

val_dict = {}
test_dict = {}
train_dict = {}

# sup_tr_f = open(sup_tr_fn, 'w')

for tup in x_train:
    # sup_tr_f.write(f"{tup[0]} {tup[1]} {tup[2]}\n")

    if tup[2] not in train_dict.keys():
        train_dict[tup[2]] = [tup]
    else:
        train_dict[tup[2]].append(tup)

# sup_tr_f.close()

te_dict = {}

for tup in x_test:
    wid, img_path = tup[2], tup[0]
    print(wid)

    if wid not in te_dict.keys():
        te_dict[wid] = [tup]
    else:
        te_dict[wid].append(tup)


# sup_val_f = open(sup_val_fn, 'w')
# sup_te_f = open(sup_te_fn, 'w')

for wid in te_dict.keys():
    split_idx = len(te_dict[wid]) // 2
    val_dict[wid] = te_dict[wid][:split_idx]
    test_dict[wid] = te_dict[wid][split_idx:]

    for idx in range(0, split_idx):
        # sup_val_f.write(te_dict[wid][idx][0] + ' ' + te_dict[wid][idx][1] + ' ' + te_dict[wid][idx][2] + '\n')
        x_val.append(te_dict[wid][idx])

    for idx in range(split_idx, len(te_dict[wid])):
        # sup_te_f.write(te_dict[wid][idx][0] + ' ' + te_dict[wid][idx][1] + ' ' + te_dict[wid][idx][2] + '\n')
        x_new_test.append(te_dict[wid][idx])
    
# sup_val_f.close()
# sup_te_f.close()

21906 9389 21906 9389
527
233
273
150
337
128
545
064
204
587
273
580
321
133
242
000
513
121
124
334
212
276
220
152
339
569
129
203
548
313
199
599
124
282
153
073
550
112
343
121
585
124
269
174
599
173
651
332
308
079
210
039
598
344
125
546
609
654
128
520
603
330
208
540
296
314
038
154
333
074
212
664
289
241
575
025
165
198
090
280
000
244
014
107
208
552
336
002
202
191
000
049
340
155
583
336
205
629
248
249
121
272
549
159
587
317
000
281
315
180
146
210
300
128
177
132
000
090
213
118
312
342
234
289
315
000
341
338
642
335
150
334
551
205
334
612
128
163
582
551
073
314
117
582
133
670
291
335
114
288
548
180
340
299
222
161
179
320
209
341
151
325
243
004
244
173
091
552
588
113
000
553
028
051
151
587
191
330
114
310
562
209
571
592
085
209
196
241
613
193
544
175
083
334
198
332
174
153
202
234
335
323
548
108
067
013
038
131
276
000
123
336
588
093
205
205
130
671
174
081
000
092
584
016
551
123
049
240
016
344
286
301
141
021
337
251
199
153
662
196
176
150
588
013
20

In [10]:
character_freq_dict = {charset_base[idx] : 0 for idx in range(len(charset_base))}

for tup in x_train:
    mg_path, gt_label, wid = tup[0], tup[1], tup[2]
    for char in list(gt_label):
        character_freq_dict[char] += 1

total = sum(character_freq_dict.values())

for key in character_freq_dict.keys():
    character_freq_dict[key] /= total

print(character_freq_dict)
char_weights = [character_freq_dict[char] for char in charset_base]


{'a': 0.07792313601340814, 'b': 0.01614329942503317, 'c': 0.019576805791568706, 'd': 0.03783840405968482, 'e': 0.12552666495961265, 'f': 0.024232407644498244, 'g': 0.017330477897530203, 'h': 0.07180101957680579, 'i': 0.059067948509043504, 'j': 0.001117344444703089, 'k': 0.007355850927628669, 'l': 0.03815265718475756, 'm': 0.02161363160222538, 'n': 0.06350240927395889, 'o': 0.07850508624502432, 'p': 0.013699108452245165, 'q': 0.0006285062501454876, 'r': 0.054947740869200866, 's': 0.05921925556926372, 't': 0.09329826113270794, 'u': 0.02597825833934682, 'v': 0.008240415279685281, 'w': 0.02607137037640541, 'x': 0.0013035685188202706, 'y': 0.01944877674061314, 'z': 0.0004190041667636584, 'A': 0.0023394399310970927, 'B': 0.0026536930561698365, 'C': 0.0014315975697758328, 'D': 0.0011522614586000606, 'E': 0.0012453734956586512, 'F': 0.0013734025466142137, 'G': 0.0015363486114667473, 'H': 0.002607137037640541, 'I': 0.002257966898670826, 'J': 0.0005121162038222491, 'K': 0.0003957261574990107, 'L

In [11]:
transforms_ = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])

In [39]:
def copy_style_imgs_wid(single_img, wid, wid_dict):
    """Obtain different same-style images based on the wid"""
    style_paths = wid_dict[wid]
    copy_paths = random.choices(style_paths, k=25 - 1)

    final_img = [single_img[:]]

    for path in copy_paths:
        path = os.path.join("..", path[0])
        new_img = 255 - cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        new_img = transforms_(new_img)
        final_img = final_img + [new_img]

    final_img = np.stack(final_img, axis=0)

    return np.asarray(final_img, dtype="float32")

In [53]:
def get_nonesense_label():
    word_len = random.randint(0, 10)

    if word_len == 0:
        return ''

    nonesense_label = random.choices(charset_base, weights=char_weights, k=word_len)
    return ''.join(nonesense_label)

In [14]:
def random_insertion(text):
    idx = random.randint(0, len(text))
    char_in = random.choices(charset_base, weights=char_weights, k=1)
    return text[:idx] + char_in[0] + text[idx:]

In [15]:
def random_deletion(text):
    idx = random.randint(0, len(text) - 1)
    return text[:idx] + text[idx + 1:]

In [16]:
def random_swap(text):
    text = list(text)
    indices = random.choices([idx for idx in range(len(text))], k=2)
    temp = text[indices[0]]
    text[indices[0]] = text[indices[1]]
    text[indices[1]] = temp
    return ''.join(text)

In [17]:
def random_replacement(text):
    idx = random.randint(0, len(text) - 1)
    new_char = random.choices(charset_base, weights=char_weights, k=1)
    text = list(text)
    text[idx] = new_char[0]
    return ''.join(text)

In [18]:
def random_edits(text):
    edit_funcs = [random_insertion, random_deletion, random_swap, random_replacement]
    edits = random.choices(edit_funcs, k=3)
    text = edits[0](text)
    text = edits[1](text)
    return text

In [78]:
def random_multi_insert(text):
    if len(text) > 2:
        num_pos = random.randint(1, 2)
    else:
        num_pos = 1
    for i in range(num_pos):
        if len(text) == 0:
            text = random.choices(charset_base, weights=char_weights, k=1)[0]
        idx = random.randint(0, len(text))
        char_in = text[-1] if idx == len(text) else text[idx]
        seq_in = char_in * random.randint(2, 5)
        text = text[:idx] + seq_in + text[idx:]
    return text[:25] if len(text) > 25 else text



In [76]:

def generate_data(dataset, wid_ids, fn, partition, wid_dict):
    random.seed(42)
    directory = '//'.join(fn.split('/')[:-1])
    nonesense_dir = directory + '/nonesense/'
    edits_dir =  directory + '/edits/'
    same_dir = directory + f'/same/'
    print(os.listdir(directory))
    print(same_dir)
    
    file = open(fn, 'w')

    os.makedirs(nonesense_dir, exist_ok=True)
    os.makedirs(edits_dir, exist_ok=True)
    os.makedirs(same_dir, exist_ok=True)

    # idx = 0
    # pairs = []
    for tup in dataset:
        inputs = []
        labels = []
        img_path, gt_label, wid = tup[0], tup[1], tup[2]

        img_path = os.path.join("..", img_path)

        img = 255 - cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = transforms_(img)
        inputs.append(copy_style_imgs_wid(img, wid, wid_dict))
        labels.append(tokenizer.encode(gt_label))


        wid_diff = wid
        while wid_diff == wid:
            wid_diff = random.choice(wid_ids)

        inputs.append(copy_style_imgs_wid(img, wid_diff, wid_dict))
        labels.append(tokenizer.encode(gt_label))

        if random.random() <= 0.5:
            rnd_sample = random.choice(dataset)[1]
            while rnd_sample[1] == gt_label:
                rnd_sample = random.choice(dataset)[1]
            nonesense_label = rnd_sample[:]
        else:
            nonesense_label = get_nonesense_label()

            if random.random() <= 0.25:
                nonesense_label = random_multi_insert(nonesense_label)
                
        inputs.append(copy_style_imgs_wid(img, wid, wid_dict))
        labels.append(tokenizer.encode(nonesense_label))
    
        edit_label = random_edits(gt_label)
        while edit_label == gt_label:
            edit_label = random_edits(gt_label)
        
        if random.random() <= 0.25:
            edit_label = random_multi_insert(edit_label)
        inputs.append(copy_style_imgs_wid(img, wid, wid_dict))
        labels.append(tokenizer.encode(edit_label))

        inputs = torch.from_numpy(np.array(inputs)).squeeze(2).cuda()

        labels = torch.from_numpy(np.array(labels)).cuda().long()
        labels = torch.nn.functional.one_hot(labels, 56).float()
        print(gt_label, nonesense_label, edit_label)

        synth_imgs = gen_model(inputs, labels).detach()
        # print(torch.min(synth_imgs), torch.max(synth_imgs))
        synth_imgs = ((synth_imgs * 0.5) + 0.5) * 255
        synth_imgs = synth_imgs.cpu().numpy()
        # print(torch.min(synth_imgs), torch.max(synth_imgs))

        new_img_path = img_path.split('.png')[0] # remove '.png
        img_name = new_img_path.split('\\')[-1]
        
        cv2.imwrite(same_dir + img_name + '_sames.png', synth_imgs[0][0])
        file.write(f"{img_path} {same_dir}{img_name}_sames.png 1 {gt_label} {gt_label} \n")

        cv2.imwrite(same_dir + img_name + '_samed.png', synth_imgs[1][0])
        file.write(f"{img_path} {same_dir}{img_name}_samed.png 1 {gt_label} {gt_label} \n")

        cv2.imwrite(nonesense_dir + img_name + '_nonesense.png', synth_imgs[2][0])
        file.write(f"{img_path} {nonesense_dir}{img_name}_nonesense.png 0 {gt_label} {nonesense_label}\n")

        cv2.imwrite(edits_dir + img_name + '_edits.png', synth_imgs[3][0])
        file.write(f"{img_path} {edits_dir}{img_name}_edits.png 0 {gt_label} {edit_label}\n")


    file.close()

In [ ]:
def prep_data_multiproc(data, num_processes):
    new_data = []
    size = len(data) // num_processes
    for i in range(num_processes ):
        start_idx = i*size
        if i == num_processes - 1:
            new_data.append(data[start_idx:])
        else:
            new_data.append(data[start_idx:start_idx+size])
    return new_data


In [37]:
for wid in train_dict.keys():
    if wid in val_dict.keys():
        val_dict[wid] += train_dict[wid]
    if wid in test_dict.keys():
        test_dict[wid] += train_dict[wid]

In [79]:
wid_ids_keys = [key for key in wids_dict.keys()]

tr_data_fn = "../../data/iam_gan/words/sia_ground_truth_train_filtered.txt"
generate_data(x_train, wid_ids_keys, tr_data_fn, 'train', train_dict)

test_data_fn = "../../data/iam_gan/words/sia_ground_truth_test_filtered.txt"
generate_data(x_val, wid_ids_keys, test_data_fn, 'test', test_dict)

valid_data_fn = "../../data/iam_gan/words/sia_ground_truth_valid_filtered.txt"
generate_data(x_new_test, wid_ids_keys, valid_data_fn, 'valid', val_dict)

['edits', 'ground_truth_train_filtered.txt', 'ground_truth_valid_filtered.txt', 'nonesense', 'same', 'sia_ground_truth_train_filtered.txt', 'superHTR.ground_truth_test_filtered.txt', 'superHTR.ground_truth_train_filtered.txt', 'superHTR.ground_truth_valid_filtered.txt', 'train', 'valid', 'valid_gen']
..//..//data//iam_gan//words/same/
jump he ju
of by effff
the new hhhhhh
or rwseiywo mo
Table been Tal
for Parana foooo
men tedededato m
That helped Thho
was back salllll
few is fewr
bar oasFbnhke mhr
of hnysavne b
lying was ying
drawn Gmmwn ardwan
why ynfbe wwwwh
for om rof
of iw fon
as hours sh
has between shav
days w eys
show uInehhhhhhhhhhhdfl sow
church would hhhhhhhhhhhurc
Jamaica be TJrmaiiiica
sex ahscgtd sxe
with ttttttearnnnnn eith
Fill e oil
when  whryen
new h nHe
cure s cuers
dashed nofrl daseh
Bill habits oBlilll
not uedrLtcvln on
an the 
and sAAAAf aubd
Megan ofwh neegaM
staff cannot stfo
the oe h
his eyes ns
of at odm
the gefodtttttttttt t
God of oooowd
gladly ttaons gliyld


In [ ]:
# tr_data_fn = "../../data/iam/words/ground_truth_train_filtered.txt"

# with open(tr_data_fn, "w") as tr_f:
#     for tup in train_data:
#         img_path, gt_label, wid = tup[0], tup[1], tup[2]
#         tr_f.write(f"{img_path} {gt_label} {wid}\n")

    

In [ ]:
# test_data_fn = "../../data/iam/words/ground_truth_test_filtered.txt"

# with open(test_data_fn, "w") as test_f:
#     for tup in test_data:
#         img_path, gt_label, wid = tup[0], tup[1], tup[2]
#         test_f.write(f"{img_path} {gt_label} {wid}\n")

# test_data_fn = "../../data/iam/words/ground_truth_test_oov_filtered.txt"

# with open(test_data_fn, "w") as test_f:
#     for tup in oov_test_data:
#         img_path, gt_label, wid = tup[0], tup[1], tup[2]
#         test_f.write(f"{img_path} {gt_label} {wid}\n")

In [ ]:
# valid_data_fn = "../../data/iam/words/ground_truth_valid_filtered.txt"

# with open(valid_data_fn, "w") as valid_f:
#     for tup in valid_data:
#         img_path, gt_label, wid = tup[0], tup[1], tup[2]
#         valid_f.write(f"{img_path} {gt_label} {wid}\n")

# valid_data_fn = "../../data/iam/words/ground_truth_valid_oov_filtered.txt"

# with open(valid_data_fn, "w") as valid_f:
#     for tup in oov_valid_data:
#         img_path, gt_label, wid = tup[0], tup[1], tup[2]
#         valid_f.write(f"{img_path} {gt_label} {wid}\n")

In [ ]:
import numpy as np

file = './test.txt'
count = 0

with open(file, 'r') as f:
    lines = f.read().splitlines()
    for line in lines:
        if len(line.split(' ')) <= 2:
            count += 1
count

4